In [ ]:
data = np.genfromtxt('loss_time_data_5000_0.001.csv', delimiter=',', skip_header=1)


# 绘制迭代次数和误差的关系图
plt.figure(figsize=(10, 6))
# plt.plot(data[:2001,0], data[:2001,1], label='Train loss', color='b')
# plt.plot(iteration_list[:2001], Train_loss_list[:2001], label='Train loss', color='r')
plt.plot(data[:,0], data[:,1], label='KAN_loss', color='b')
plt.plot(iteration_list, Train_loss_list, label='NN_loss', color='r')
plt.yscale('log')
plt.ylim(10e-12, 10e-3)
plt.xlabel('Iteration')
plt.ylabel('Loss')
# plt.title('Loss vs. Iteration')
plt.legend()
plt.grid(True)
# plt.show()
plt.savefig('Cubic2D_Error_5000_0.001.png', format='png', dpi=600, bbox_inches='tight')

In [ ]:
import numpy as np
from scipy.optimize import linprog

# Define dimensions
rows, cols = 5, 21

# Define the row and column sums
row_sums = [20, 5, 50, 20, 5]
col_sums = [10, 4, 6, 4, 4, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7, 12]

# Define the bounds for each element in the matrix (positive integers)
bounds = [(1, None)] * (rows * cols)

# Linear programming formulation
c = [0] * (rows * cols)  # Objective: minimize 0 to just find feasible solution
A_eq = []
b_eq = row_sums + col_sums

# Row constraints
for i in range(rows):
    row_constraint = [1 if j // cols == i else 0 for j in range(rows * cols)]
    A_eq.append(row_constraint)

# Column constraints
for j in range(cols):
    col_constraint = [1 if j % cols == i else 0 for i in range(rows * cols)]
    A_eq.append(col_constraint)

# Solve the linear programming problem
res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Extract solution if feasible
if res.success:
    matrix = np.array(res.x).reshape(rows, cols).round().astype(int)
else:
    matrix = None

print(matrix)


In [ ]:
# 尝试通过随机生成的方法，确保满足所有条件

def generate_matrix_with_constraints(row_sums, col_sums):
    rows, cols = len(row_sums), len(col_sums)
    matrix = np.zeros((rows, cols))
    
    # 初始矩阵填充为 0.5 以满足非负限制
    matrix += 0.5
    
    # 调整行列的剩余部分以满足约束
    for i in range(rows):
        row_target = row_sums[i] - matrix[i].sum()
        for j in range(cols):
            col_target = col_sums[j] - matrix[:, j].sum()
            add_value = min(row_target, col_target)
            matrix[i, j] += add_value
            row_target -= add_value
            col_sums[j] -= add_value
            if row_target <= 0:
                break

    return matrix

# 使用新方法生成矩阵
new_matrix = generate_matrix_with_constraints(row_sums, col_sums)

# 验证行列和是否满足要求
new_row_sums = new_matrix.sum(axis=1)
new_col_sums = new_matrix.sum(axis=0)

new_matrix, new_row_sums, new_col_sums


In [ ]:
# 测试误差
test_steps = 1000
# test_np = np.random.uniform(0, 2, test_steps)
test_np = np.linspace(0, 1, num=test_steps)
y_test_np1 = odeint(f, y0, test_np)[:,0] 
y_test_np2 = odeint(f, y0, test_np)[:,1]
y_test_np = torch.stack([y_test_np1, y_test_np2], axis=1)
# y_test_new = torch.cat((y_test_np1, y_test_np2), dim=0)
f_test_1 =sol_fun_1(y_test_np)
f_test_2 =sol_fun_2(y_test_np)
f_test = torch.cat((f_test_1, f_test_2), dim=1).detach()
f_nn= model(y_test_np).detach()
Test_loss =  torch.mean((f_test - f_nn)**2)


In [3]:
import torch
import torch.nn.functional as F
from BKAN import KANLayer
torch.set_default_dtype(torch.float64) # 设置默认张量数据类型为双精度浮点数
y_np = np.linspace(0, 1, num=5000)
y_train = np.stack((y_np, y_np), axis=1)
y_train_new  = torch.tensor(y_train)
model = KANModel(width=[2, 5, 2])
# KANModel.act_fun
# model(y_train_new)
# print(model.act_fun[0](y_train_new))
# x = y_train_new.view(-1, 2)
# model.act_fun[0](y_train_new)

In [ ]:
y_train_new  = torch.tensor(y_train)
batch = y_train_new.shape[0]
x = torch.einsum('ij,k->ikj', y_train_new, torch.ones(5)).reshape(batch, 10).permute(1, 0)
preacts = x.permute(1, 0).clone().reshape(batch, 5, 2)
# y = F.linear(SplineLinearLayer._compute_b_splines(y_train_new).view(y_train_new.size(0), -1),
#                                  SplineLinearLayer._scaled_spline_weights.view(5, -1))
# y = y.permute(1, 0)

In [1]:
import torch
import torch.nn.functional as F
# import torch.optim as optim
import torch.nn as nn


class SplineLinearLayer(torch.nn.Module):
    def __init__(self, input_dim, output_dim, num_knots=5, spline_order=3,
                 spline_scale=1.0, activation=torch.nn.SiLU, grid_epsilon=0.02, grid_range=[-1, 1],
                 standalone_spline_scaling=True):
        super(SplineLinearLayer, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_knots = num_knots
        self.spline_order = spline_order
        self.grid_epsilon = grid_epsilon
        self.grid_range = grid_range
        self.standalone_spline_scaling = standalone_spline_scaling

        self.knots = self._calculate_knots(grid_range, num_knots, spline_order)
        self.base_weights = torch.nn.Parameter(torch.Tensor(output_dim, input_dim))
        self.spline_weights = torch.nn.Parameter(torch.Tensor(output_dim, input_dim, num_knots + spline_order))

        # 是否添加B样条函数前面的权重
        if standalone_spline_scaling:
            self.spline_scales = torch.nn.Parameter(torch.Tensor(output_dim, input_dim))

        # self.noise_scale = noise_scale
        # self.base_scale = base_scale
        self.spline_scale = spline_scale
        self.activation = activation()

        self._initialize_parameters()


    # 初始化参数
    def _initialize_parameters(self):
        torch.nn.init.xavier_uniform_(self.base_weights, gain=torch.sqrt(torch.tensor(2.0)))
        noise = torch.rand(self.num_knots + 1, self.input_dim, self.output_dim) - 0.5
        self.spline_weights.data.copy_(self.spline_scale * self._initialize_spline_weights(noise))
        if self.standalone_spline_scaling:
            torch.nn.init.xavier_uniform_(self.spline_scales, gain=torch.sqrt(torch.tensor(2.0)))


    # 将一维的节点扩展到高维上 并加扩展
    def _calculate_knots(self, grid_range, num_knots, spline_order):
        h = (grid_range[1] - grid_range[0]) / num_knots
        knots = torch.arange(-spline_order, num_knots + spline_order + 1) * h + grid_range[0]
        return knots.expand(self.input_dim, -1).contiguous()


    # 输入时间格点和噪音，求的是噪音和B样条在这些点上的最小二乘解
    def _initialize_spline_weights(self, noise):
        return self._fit_curve_to_coefficients(self.knots.T[self.spline_order : -self.spline_order], noise)
    # self.knots.T[self.spline_order : -self.spline_order] 将扩充的节点变回去


    # B样条函数的计算
    def _compute_b_splines(self, x):
        x = x.unsqueeze(-1)
        bases = ((x >= self.knots[:, :-1]) & (x < self.knots[:, 1:])).to(x.dtype)
        for k in range(1, self.spline_order + 1):
            bases = ((x - self.knots[:, : -(k + 1)]) / (self.knots[:, k:-1] - self.knots[:, : -(k + 1)]) * bases[:, :, :-1] +
                     (self.knots[:, k + 1 :] - x) / (self.knots[:, k + 1 :] - self.knots[:, 1:(-k)]) * bases[:, :, 1:])
        return bases.contiguous()

    # 求
    def _fit_curve_to_coefficients(self, x, y):
        A = self._compute_b_splines(x).transpose(0, 1)
        B = y.transpose(0, 1)
        solution = torch.linalg.lstsq(A, B).solution
        return solution.permute(2, 0, 1).contiguous()

    @property
    def _scaled_spline_weights(self):
        return self.spline_weights * (self.spline_scales.unsqueeze(-1) if self.standalone_spline_scaling else 1.0)

    def forward(self, x):
        base_output = F.linear(self.activation(x), self.base_weights)
        spline_output = F.linear(self._compute_b_splines(x).view(x.size(0), -1),
                                 self._scaled_spline_weights.view(self.output_dim, -1))
        
        # batch = x.shape[0]
        # x = torch.einsum('ij,k->ikj', x, torch.ones(self.out_dim, device=self.device)).reshape(batch, self.size).permute(1, 0)
        # preacts = x.permute(1, 0).clone().reshape(batch, self.out_dim, self.in_dim)
        y = base_output + spline_output
        return y #,preacts
        # return spline_output

    @torch.no_grad()
    def _update_knots(self, x, margin=0.01):
        batch = x.size(0)
        splines = self._compute_b_splines(x).permute(1, 0, 2)
        orig_coeff = self._scaled_spline_weights.permute(1, 2, 0)
        unreduced_spline_output = torch.bmm(splines, orig_coeff).permute(1, 0, 2)

        x_sorted = torch.sort(x, dim=0)[0]
        adaptive_knots = x_sorted[torch.linspace(0, batch - 1, self.num_knots + 1, dtype=torch.int64, device=x.device)]

        uniform_step = (x_sorted[-1] - x_sorted[0] + 2 * margin) / self.num_knots
        uniform_knots = torch.arange(self.num_knots + 1, dtype=torch.float32, device=x.device).unsqueeze(1) * uniform_step + x_sorted[0] - margin

        knots = self.grid_epsilon * uniform_knots + (1 - self.grid_epsilon) * adaptive_knots
        knots = torch.cat([
            knots[:1] - uniform_step * torch.arange(self.spline_order, 0, -1, device=x.device).unsqueeze(1),
            knots,
            knots[-1:] + uniform_step * torch.arange(1, self.spline_order + 1, device=x.device).unsqueeze(1),
        ], dim=0)

        self.knots.copy_(knots.T)
        self.spline_weights.data.copy_(self._fit_curve_to_coefficients(x, unreduced_spline_output))


class KANLayer(nn.Module):
    def __init__(self, in_features, out_features, 
                 num_knots=10, spline_order=3, 
                 spline_scale=1.0, activation=nn.SiLU, 
                 grid_epsilon=0.02, grid_range=[-1, 1]):
        super(KANLayer, self).__init__()
        self.linear = SplineLinearLayer(in_features, out_features, num_knots, 
                                        spline_order,
                                        spline_scale, activation, grid_epsilon, grid_range)
    def forward(self, x):
        return self.linear(x)

In [7]:
model = KANLayer(2,5)
import torch
import numpy as np
torch.set_default_dtype(torch.float64) # 设置默认张量数据类型为双精度浮点数
y_np = np.linspace(0, 1, num=5000)
y_train = np.stack((y_np, y_np), axis=1)
y_train_new  = torch.tensor(y_train)

model(y_train_new).shape

torch.Size([5000, 5])